This tutorial is automatically generated from the file test/python/tutorials//TestPythonBuildVesselNetworkTutorial.py.



In [ ]:
# Jupyter notebook specific imports 
import matplotlib as mpl 
from IPython import display 
%matplotlib inline


## Introduction
This tutorial introduces modelling vessel networks. It will cover the following techniques:

 * Building a network from a collection of nodes, segments and vessels
 * Writing networks to file and visualizing it
 * Building a network using a network generator
 
## The Test



In [ ]:
import chaste # Core Chaste functionality
import microvessel_chaste # Core Microvessel Chaste functionality
import microvessel_chaste.population.vessel # Vessel tools
import microvessel_chaste.visualization # Visualization
from microvessel_chaste.utility import * # Dimensional analysis: bring in all units for convenience

## Test 1 - Building a vessel network manually, writing it to file and visualizing it
In the first test we will build a vessel network from its constituent components; nodes,
segments and vessels. We will do some simple tests to make sure the network has been formed
as expected. Then we write the network to file and visualize it.



First we make some nodes, which are point features from which vessels can be constructed.
They are initialized with a location. Vessel network components are specialized (templated)
over spatial dimension, and can be 2D or 3D. We will create a Y shaped network.
Later we will learn how to build up networks in a more efficient manner. Note that we
are being explicit regarding units, setting a length scale of 1 micron.



In [ ]:
file_handler = chaste.core.OutputFileHandler("Python/TestPythonBuildVesselNetworkLiteratePaper", True)
length_scale = 1.e-6*metre()
length = 100.0
n1 = microvessel_chaste.population.vessel.VesselNode3(0.0, 0.0 ,0.0, length_scale)
n2 = microvessel_chaste.population.vessel.VesselNode3(length, 0.0, 0.0, length_scale)
n3 = microvessel_chaste.population.vessel.VesselNode3(2.0 * length, length, 0.0, length_scale)
n4 = microvessel_chaste.population.vessel.VesselNode3(2.0 * length, -length, 0.0, length_scale)

Next we make vessel segments and vessels. Vessel segments are straight-line features
which contain a vascular node at each end. Vessels can be constructed from multiple vessel segments,
but in this case each vessel just has a single segment.



In [ ]:
v1 = microvessel_chaste.population.vessel.Vessel3([n1 ,n2])
v2 = microvessel_chaste.population.vessel.Vessel3([n2, n3])
v3 = microvessel_chaste.population.vessel.Vessel3([n2, n4])

Now we can add our vessels to a vessel network.



In [ ]:
network = microvessel_chaste.population.vessel.VesselNetwork3.Create()
network.AddVessel(v1)
network.AddVessel(v2)
network.AddVessel(v3)
network.SetSegmentRadii(10.0*length_scale)
network.SetNodeRadii(10.0*length_scale)

We can visualize the network



In [ ]:
scene = microvessel_chaste.visualization.MicrovesselVtkScene3()
scene.SetVesselNetwork(network)
nb_manager = microvessel_chaste.visualization.JupyterNotebookManager()
nb_manager.vtk_show(scene, height=600)

Next we write out network to file. We use the Chaste `OutputFileHandler` functionality to manage
the output location. Networks are written using VTKs PolyData format, which should have a .vtp extension.



In [ ]:
writer = microvessel_chaste.population.vessel.VesselNetworkWriter3()
writer.SetVesselNetwork(network)
writer.SetFileName(file_handler.GetOutputDirectoryFullPath() + "bifurcating_network.vtp")
writer.Write()

We can visualize then network in Paraview.

## Test 2 - Building a vessel network with a generator
In the first test we manually built a network from its components. This is tedious. We can use a generator
instead.



Create a hexagonal network in 3D space using a generator. Specify the target network
width and height and the desired vessel length. The use of dimensional analysis is demonstrated
by now using a fictitious 'cell width' reference length unit instead of microns.



In [ ]:
file_handler = chaste.core.OutputFileHandler("Python/TestPythonBuildVesselNetworkLiteratePaperGenerator", True)
cell_width = 25.e-6 * metre()
BaseUnits.Instance().SetReferenceLengthScale(cell_width)
target_width = 60.0 * cell_width
target_height = 30.0 * cell_width
vessel_length = 4.0 * cell_width
network_generator = microvessel_chaste.population.vessel.VesselNetworkGenerator3()
network = network_generator.GenerateHexagonalNetwork(target_width, target_height, vessel_length)

We can visualize the network



In [ ]:
scene = microvessel_chaste.visualization.MicrovesselVtkScene3()
scene.SetVesselNetwork(network)
nb_manager.vtk_show(scene, height=600)

We write the network to file as before. We want to over-ride the
reference length scale so that the output is written in micron.



In [ ]:
writer = microvessel_chaste.population.vessel.VesselNetworkWriter3()
writer.SetFileName(file_handler.GetOutputDirectoryFullPath() + "hexagonal_network.vtp")
writer.SetVesselNetwork(network)
micron_length_scale = 1.e-6 * metre()
writer.SetReferenceLengthScale(micron_length_scale)
writer.Write()

Use a reader to read the network back in from the VTK file. Our network was written in units of micron, so
we need to tell the reader this so that locations are suitably stored.



In [ ]:
network_reader = microvessel_chaste.population.vessel.VesselNetworkReader3()
network_reader.SetReferenceLengthScale(micron_length_scale)
network_reader.SetFileName(file_handler.GetOutputDirectoryFullPath() + "hexagonal_network.vtp")
network_from_file = network_reader.Read()

Again, we can visualize the network



In [ ]:
scene = microvessel_chaste.visualization.MicrovesselVtkScene3()
scene.SetVesselNetwork(network)
nb_manager.vtk_show(scene, height=600)